<a href="https://colab.research.google.com/github/niapollard/Benchmarking_QC/blob/main/Al_reduced_hamiltonian_vqe_%26_numpy_total_energy_SIMULATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q jarvis-tools qiskit==0.43.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 31.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
!pip install qiskit_nature==0.6.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 37.8 MB/s eta 0:00:00


In [3]:
!pip install pyscf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 MB 11.8 MB/s eta 0:00:00


In [4]:
!pip install qiskit-aer==0.12.0

In [5]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *

In [6]:
from qiskit import Aer
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.drivers.second_quantization import MethodType

In [7]:
import time

start_time = time.time()

In [8]:
molecule = Molecule(
    geometry=[['Al',[0.00000000, 0.00000000, 0.00000000]]],
    charge=-1,
    multiplicity=1)
# multiplicity has to be that active space is even therefore should always be 1 (2s+1)
# charge has to be that active space is even therefore species with an odd number of atoms should have a charge of -1
MethodType = MethodType.RKS

driver = ElectronicStructureMoleculeDriver(
    molecule, basis="lanl2dz", method=MethodType, driver_type=ElectronicStructureDriverType.PYSCF
)

properties = driver.run()
problem = ElectronicStructureProblem(driver)
qubit_converter = QubitConverter(JordanWignerMapper())

In [9]:
second_q_ops = problem.second_q_ops()
second_q_ops = list(second_q_ops.values()) #list of second quantized operators
hamiltonian = second_q_ops[0] #Hamiltonian is the first term
print(hamiltonian)

Fermionic Operator
register length=16, number terms=16
  (1+0j) * ( +_0 -_0 )
+ (1+0j) * ( +_1 -_1 )
+ (1+0j) * ( +_2 -_2 )
+ (1+0j) * ( +_3 -_3 )
+ (1+0j) * ( +_4 -_4 )
+ (1+0j) * ( +_5 -_5 )
+ (1+0j) * ( +_6 -_6 )
+ (1+0j) * ( +_7 -_7 )
+ (1+0j) * ( ...


In [10]:
#lists the full hamiltonian
#hamiltonian.to_list()

In [11]:
from qiskit.algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()

<ipython-input-11-d9d9d0820b7f>:3: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


In [12]:
tmp = properties.get_property('ParticleNumber')
alpha_occ=tmp.occupation_alpha
beta_occ=tmp.occupation_beta

In [13]:
alpha_occ

array([1., 1., 1., 1., 1., 1., 1., 0.])

In [14]:
import numpy as np
mo_considered=3
#active and inactive space has to be even, non-magnetic

first_index = min(np.where(alpha_occ<1)[0][0],np.where(beta_occ<1)[0][0])

orb_act = np.arange(first_index-mo_considered,first_index)+1
orb_act

array([5, 6, 7])

In [15]:
from qiskit_nature.transformers.second_quantization.electronic.active_space_transformer import ActiveSpaceTransformer

#orb_act   = []
transformer= ActiveSpaceTransformer(num_electrons=mo_considered+1, num_molecular_orbitals=len(orb_act), active_orbitals=orb_act)
problem_reduced = ElectronicStructureProblem(driver, [transformer])
second_q_ops_reduced = problem_reduced.second_q_ops()

second_q_ops_reduced = list(second_q_ops_reduced.values())
hamiltonian_reduced = second_q_ops_reduced[0]
print(hamiltonian_reduced)

Fermionic Operator
register length=6, number terms=6
  (1+0j) * ( +_0 -_0 )
+ (1+0j) * ( +_1 -_1 )
+ (1+0j) * ( +_2 -_2 )
+ (1+0j) * ( +_3 -_3 )
+ (1+0j) * ( +_4 -_4 )
+ (1+0j) * ( +_5 -_5 )


In [16]:
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper)
qubit_op_jw = converter.convert(hamiltonian_reduced)
print(qubit_op_jw) #weighted sum of Pauli terms

3.0 * IIIIII
- 0.5 * IIIIIZ
- 0.5 * IIIIZI
- 0.5 * IIIZII
- 0.5 * IIZIII
- 0.5 * IZIIII
- 0.5 * ZIIIII


In [17]:
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
import itertools, functools
from qiskit.opflow import I, X, Y, Z
from jarvis.db.figshare import get_wann_electron, get_wann_phonon, get_hk_tb
from jarvis.core.circuits import QuantumCircuitLibrary

In [18]:
seed = 42
counts = []
values = []


def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)
n_qubits = qubit_op_jw.num_qubits
ansatz = QuantumCircuitLibrary(n_qubits=n_qubits, reps=1).circuit6()
slsqp = SLSQP(maxiter=1000)
vqe = VQE(ansatz, optimizer=slsqp, quantum_instance=qi,callback=store_intermediate_result)

result = vqe.compute_minimum_eigenvalue(operator=qubit_op_jw)


eigenvalue = result.eigenvalue
#print(result)
print('VQE eig.', eigenvalue)

<ipython-input-18-7f1824667f0f>:11: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)
<ipython-input-18-7f1824667f0f>:15: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=slsqp, quantum_instance=qi,callback=store_intermediate_result)


VQE eig. (8.5066017987e-08+0j)


In [19]:
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver

solver =GroundStateEigensolver(converter, vqe)
result=solver.solve(problem_reduced)
print(result)

<ipython-input-19-fb24484ecdc7>:1: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
<ipython-input-19-fb24484ecdc7>:1: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
<ipython-input-19-fb24484ecdc7>:3: NatureDeprecationWarning: The qiskit_nature.algorithms.ground_state_solvers.GroundStateEigensolver class is deprecated as of version 0.5.0 and will be removed no sooner t

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -42.440577255973
  - computed part:      -1.174195669899
  - ActiveSpaceTransformer extracted energy part: -41.266381586074
~ Nuclear repulsion energy (Hartree): 0.0
> Total ground state energy (Hartree): -42.440577255973
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.618 S^2: 1.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.0]
    - computed part:      [0.0  0.0  0.0]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


In [20]:
solver =GroundStateEigensolver(converter, numpy_solver)
result=solver.solve(problem_reduced)
print(result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -42.454153629561
  - computed part:      -1.19046604944
  - ActiveSpaceTransformer extracted energy part: -41.263687580121
~ Nuclear repulsion energy (Hartree): 0.0
> Total ground state energy (Hartree): -42.454153629561
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 1.000 S^2: 2.000 M: 0.007
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.0]
    - computed part:      [0.0  0.0  0.0]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


In [21]:
# Record the end time
end_time = time.time()

# Calculate the duration
duration = end_time - start_time

# Print the duration
print("Job took {:.2f} seconds to run.".format(duration))

Job took 30.93 seconds to run.
